# I. Introduction

### Final Project

- Study Group 4 Hacktiv8, Batch 011 Full Time Data Science
- Our team :
  - Megatheo J. Rumahorbo
  - Aditya Vianseto
  - Hafiz Febry
  - Nabila Edelliana K

## Objectives
This Project aims to create a user-friendly chatbot (GitCoff" for e-commerce "GitCoffee" to help communicating with customers in the more effective way for non-technical question with the concept of machine learning and NLP.

## Dataset
We build our own dataset in order to have a relevan dataset and in to complete the aims of the objectives.
1. Intents : contains 3 informations, dataset for the chatbot
    - tag : defines the class of the question/input and the answer/output.
    - pattern : the data that represent the input for GitCoff from user/customer and will be trained later to the model.
    - responses : defines the responses that will be selected randomly as the output for the input.
2. data_kopi.csv : contains 5 informations, dataset for recommeder system.

# II. Import Library

In [25]:
# Import library that will be used in this project

# Library for preprocessing
import string
import re
import numpy as np
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Library for modelling
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from util import JSONParser
from sklearn.model_selection import train_test_split

# Library for model evaluation
from sklearn import metrics

# Data Loading

In [26]:
# Load data from folders
path = "dataset/intents.json"

# Defining JSONParser
jp = JSONParser()

# Parsing data intents
jp.parse(path)

# Building dataframe and save it to variable df
df = jp.get_dataframe()

In [27]:
df

,text_input,intents
0,Hai,menyapa
1,Halo,menyapa
2,Hei,menyapa
3,Oi,menyapa
4,Woy,menyapa
...,...,...
1028,Kopi yang paling mahal dijual disini ada gak sih?,kopi_mahal
1029,jenis kopi yang harganya paling mahal ada gak?...,kopi_mahal
1030,Punya kopi yang harganya paling mahal kan,kopi_mahal
1031,Punya kopi termahal kan? kasih tau aku bisa?,kopi_mahal


There are 1003 data with 2 columns. Note that will be using `intents` as our target when modelling the data.

# III. Data Preprocessing

### 3.1. Data Cleaning

In [28]:
# Look into sample chat
sample_chat = df.text_input[500]
sample_chat

'Produk yang dijual apa aja ya? banyak ga?'

It looks like we need some data cleaning because we want "Apa bisa batalin pesanan?" has no difference with "apa bisa batalin pesanan" as the model will read the data in case-sensitive.

In [29]:
# Transforming the chat into lowercase
chat_lower = sample_chat.lower()
chat_lower

'produk yang dijual apa aja ya? banyak ga?'

In [30]:
# Remove punctuations from the chat
chat_punct = chat_lower.translate(str.maketrans("","",string.punctuation))
chat_punct

'produk yang dijual apa aja ya banyak ga'

In [31]:
# We need to change non-alphabetical characters with spaces too to make the data cleaner.
chat_punct = re.sub("[^A-Za-z\s']"," ", chat_punct)
chat_punct

'produk yang dijual apa aja ya banyak ga'

In [32]:
# Remove tab, in case the customer is mistype inputting tab on their chat
chat_punct = chat_punct.strip()
chat_punct

'produk yang dijual apa aja ya banyak ga'

In [33]:
# Defining Stemmer
stemmer = StemmerFactory().create_stemmer()

# Applying stemmer to the chat
output   = stemmer.stem(chat_punct)
output

'produk yang jual apa aja ya banyak ga'

We don't remove stopword because in our data, there are much of words that is important to detect tag. For example, the word "hai" is treated as stopwords in indonesian stopwords. Since the word "hai" is in the tag "menyapa", we don't want to remove it.

In [34]:
# Stopwords remove
#stopword = StopWordRemoverFactory().create_stop_word_remover()

#output_stopword = stopword.remove(output)
#output_stopword

In [35]:
# Create function to case folding corpus in the dataframe
def document_processing(document):
    # Transform Document Into Lowercase
    document = document.lower()

    # Remove Punctuation From Document
    document = document.translate(str.maketrans("","",string.punctuation))

    # Remove Digit From Document
    document = re.sub("[^A-Za-z\s']"," ", document)

    # Remove Tab From Document
    document = document.strip()

    #Stemmer
    stemmer = StemmerFactory().create_stemmer()

    # stemming process
    document = stemmer.stem(document)

    return document

### 3.2. Data Splitting

In [36]:
# Split between data (X) and target (y)
X = df.text_input
y = df.intents

In [37]:
X_proc = X.apply(document_processing)

### 3.3. Word Vectorization

Vectorization is converting chats into vector form so that they can be used for further processing. The token length in the vector is defined from the most words in a single chat in the data.

In [38]:
vect = CountVectorizer()
vect.fit(X_proc)

CountVectorizer()

In [39]:
X_vect = vect.transform(X_proc)

In [40]:
# Split Data for Train+Validation Set and Test-Set
X_train, X_test,  y_train, y_test = train_test_split(X_proc, y,test_size=0.2, random_state=4)
print(f"Train-Set shape      : {len(X_train),len(y_train)}")
print("Test-Set shape       :",X_test.shape)

Train-Set shape      : (826, 826)
Test-Set shape       : (207,)


# IV. Model Training

Training using pipeline as a container for vectorizer and the model

In [41]:
# MultinomialNB Pipeline
nb = make_pipeline(CountVectorizer(),
                   MultinomialNB())

# Training
nb.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [42]:
# Random Forest Pipeline
rf = make_pipeline(CountVectorizer(),
                   RandomForestClassifier())

# Training
rf.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [43]:
# Decision Tree Pipeline
dt = make_pipeline(CountVectorizer(),
                   DecisionTreeClassifier())

# Training
dt.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# V. Model Evaluation

## 5.1. Model Evaluation for MultinomialNB

In [44]:
y_train_pred_nb = nb.predict(X_train)
y_test_pred_nb = nb.predict(X_test)
print("Train-Set Multinomial Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_train_pred_nb, y_train)*100)
print("Test-Set Multinomial Naive Bayes model accuracy(in %) :", metrics.accuracy_score(y_test_pred_nb, y_test)*100)

Train-Set Multinomial Naive Bayes model accuracy(in %): 97.21549636803874
Test-Set Multinomial Naive Bayes model accuracy(in %) : 80.19323671497585


## 5.2. Model Evaluation for Decision Tree

In [45]:
y_train_pred_dt = dt.predict(X_train)
y_test_pred_dt = dt.predict(X_test)
print("Train-Set Multinomial Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_train_pred_dt, y_train)*100)
print("Test-Set Multinomial Naive Bayes model accuracy(in %) :", metrics.accuracy_score(y_test_pred_dt, y_test)*100)

Train-Set Multinomial Naive Bayes model accuracy(in %): 100.0
Test-Set Multinomial Naive Bayes model accuracy(in %) : 85.5072463768116


## 5.3. Model Evaluation for Random Forest

In [46]:
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)
print("Train-Set Multinomial Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_train_pred_rf, y_train)*100)
print("Test-Set Multinomial Naive Bayes model accuracy(in %) :", metrics.accuracy_score(y_test_pred_rf, y_test)*100)

Train-Set Multinomial Naive Bayes model accuracy(in %): 100.0
Test-Set Multinomial Naive Bayes model accuracy(in %) : 85.02415458937197


It turns out that the best model goes to MultinomialNB where the accuracy for training and test set are quiate good and the model is goodfit since the difference for the accuracy in training and test is not too big.

# VI. Building Chatbot

We use the while-loop concept to build the chatbot. We set the threshold at <0.2 is for the unknown input. For example, the chat that is out of topic will be treated as unknown input.

In [47]:
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # Preprocessing
    chat_processed = document_processing(chat)
    # Intent prediction (tag)
    res = nb.predict_proba([chat_processed])
    # get the probability value and its location
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    
    # Define condition for unknown input
    print("Saya :",chat)
    if max_prob < 0.20:
        print("Bot  : Maaf Kak, aku ga ngerti")
    # Define condition to give response towards specific tag
    else:
        print(f"Bot  : {jp.get_response(nb.classes_[max_idx])}\n")
    # Define response to end the chat for tag "menutup"
    if nb.classes_[max_idx] == 'menutup':
        break
    

Anda Terhubung dengan chatbot Kami
Saya : hai
Bot  : Mari kak, ada yang saya bisa bantu?

Saya : nama kamu siapa si?
Bot  : Kakak bisa panggil aku Gitcoff ya, salam kenal

Saya : oke gitcoff, ini toko apa sih?
Bot  : Git Coffee adalah sebuah toko online yang menjadi sebuah terobosan baru dalam dunia perkopian nih kak. Disini kami menjual berbagai macam kopi siap saji mulai dari kopi arabica, robusta, luwak, latte, macchiato, dan ada juga kopi dengan berbagai rasa lainnya. Untuk info dan pemesanan produk kami, kakak bisa tanyakan disini atau kunjungi website kami di www.git-coffee.com ya kak

Saya : jualan apa aja kamu?
Bot  : Di Git Coffee kita menjual kopi dalam bentuk bubuk ya kak seperti kopi arabica, robusta, luwak, latte, macchiato, dan ada juga kopi dengan berbagai rasa lainnya, info produk secara lengkapnya bisa kakak lihat di www.git-coffee.com/produk ya kak

Saya : nanti cara bayarnya gemana?
Bot  : Untuk melakukan pembayaran kakak bisa mengikuti instruksi pada link berikut ww

It seems that our chatbot is succesfully worked. But there's several things that we want to improve :
1. Upgrade the chatbot to be flexible when user respond the answer given by the chat bot, and at the same time giving the chatbot another question/input.
2. Upgrade the chatbot to have memories over previous chat to make the chatbot to be more user-friendly.